<h3 style="text-align: center;">ESCUELA TÉCNICA SUPERIOR DE INGENIERÍA INFORMÁTICA</h3>
<h3 style="text-align: center;">UNIVERSIDAD DE SEVILLA</h3>
<h3 style="text-align: center;">MATEMÁTICA APLICADA A SISTEMAS DE INFORMACIÓN</h3>
<h3 style="text-align: center;">Tercera práctica de laboratorio</h3>
<h3>Programación lineal: Conceptos básicos</h3>
<p>En esta práctica trataremos la Programación Lineal y el método del Símplex. Hemos estudido cómo la región factible de un problema de programación lineal es un convexo, concretamente un polítopo. Gracias al Teorema de representación si $S$ es la región factible de nuestro problema descrita en formato estándar, por lo tanto $S=  \{ \mathbf{x} \in {\mathbb R}^n: \,   A \, \mathbf{x} = \mathbf{b},\, \mathbf{x} \ge \mathbf{0} \}$, con $A$ matriz $m \times n$ de rango $m$. <br />Sean $\mathbf{x}^1, \ldots, \mathbf{x}^k$ los puntos extremos de $S$, y $\vec{\mathbf{d}}^1, \ldots, \vec{\mathbf{d}}^q$ las  direcciones extremas de $S$</p>
<p>$$ \mathbf{x} \in S \Longleftrightarrow \mathbf{x} = \sum_{j=1}^k \lambda_j \, \mathbf{x}^j \, +\, \sum_{j=1}^q \, \mu_j \, \vec{\mathbf{d}}^j,\; \mbox{con} \; \lambda_j \ge 0, \, \sum_{j=1}^k \lambda_j=1, \; \mu_j \ge 0$$</p>
<p>Partamos del mismo ejemplo que hemos visto en clase de teoría:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2) \\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>En formato estándar:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2) \\ \mbox{s.t.} & - x_1 + x_2 +x_3\hspace{.77cm}=2 \\&  - x_1 + 2 x_2 \hspace{.6cm}+x_4= 6 \\&  x_j \geq 0, \; j=1,\ldots,4 \end{array} \right\}$$</p>
<p>Estamos interesados en calcular los puntos extremos de nuestra región factible, para ellos necesitaremos ciertas herramientas del Álgebra lineal, si introducimos las matriz de coeficientes $A$ y el vector $b$:</p>

In [ ]:
A=matrix([[-1,1,1,0],[-1,2,0,1]])
b=vector([2,6])

In [ ]:
A;show(A)

<p>Si estamos interesados en el rango de $A$, número de filas o columnas</p>

In [ ]:
rank(A);A.nrows();A.ncols()

<p><span id="cell_outer_23"><span id="cell_outer_14">Para resolver <span><span><span>el sistema de ecuaciones lineales Ax</span><span style="position: relative; margin-left: 0.277em;"><span>=</span></span><span style="position: relative; margin-left: 0.277em;"><span>b</span></span></span></span> disponemos, al menos, de dos formas: </span></span></p>

In [ ]:
A.solve_right(b)

In [ ]:
A\b

<p>Nuestro sistema tiene infinitas soluciones pero SAGE sólo nos proporciona una, que es además una solución básica factible con base $\{1,2\}$. Estamos interesados en calcular todos los puntos extremos, como hemos visto en teoría, para ello nos basta resolver sistemas lineales donde la matriz de coeficientes sea submatriz de $A$ con rango máximo. Veamos en primer lugar cómo seleccionar una submatriz de $A$. Teniendo en cuenta que SAGE comienza a contar en el 0, si queremos tomar las dos filas de $A$ (primera y segunda fila, SAGE lo etiqueta "[0, 1]"), y análogamente para la primera, segunda y tercera columna. Por tanto, podemos escribir</p>

In [ ]:
A[[0,1],[0,1,2]]

<p>Para seleccionar todas las filas podemos usar el comando ":" que también nos permite seleccionar un rango "i:j" que nos proporcinará los elementos $\{i,i+1,i+2,\ldots,j-2,j-1\}$, <strong>sin alcanzar</strong> j, de esta forma podemos tomar la submatriz con las columnas segunda y tercera (sin olvidar que SAGE comienza a contar en 0)</p>

In [ ]:
A[:,1:3]

In [ ]:
A[:,1:3]\b

<p>La solución del sistema anterior no verifica $B^{-1}b\geq 0$ por lo que no representa una solución básica factible.</p>
<p>Para que sea más sencillo el recorrido de todas las combinaciones de columnas de $A$ partimos de una lista con tantos elementos como columnas tiene $A$:</p>

In [ ]:
I=range(A.ncols())

In [ ]:
I

<p>La orden powerset() nos proporciona todos los subconjuntos de un conjunto, de esta forma:</p>

In [ ]:
for i in powerset(I):
    i

<p>Tan sólo nos interesan los subconjuntos de dos elementos y tales que la submatriz que definan tenga rango máximo. Así</p>

In [ ]:
for i in powerset(I):
    if len(i)==2 and (( (A[:,i]).det())!=0):
        print (A[:,i]\b, 'base=',i)

<p>Por ejemplo, el otuput "(2, 2)  base= [1, 3] " significa que para la matriz básica $B=[\mathbf{a_2}, \mathbf{a_4}]$, las variables básicas son $(x_2, x_4)=(2, 2)$. </p>

<p>Ahora queremos describir las direcciones extremas para ello nos bastaba construir la matriz:</p>
<p>$$\left( \!\!\begin{array}{c} - B^{-1}N \\ I \end{array} \!\!\right)$$</p>
<p>y de ésta quedarnos con las columnas positivas.<br /><br /></p>

In [ ]:
for i in powerset(I):
    if (len(i)==(A.nrows())) and (((A[:,i]).det())!=0):
        noti=[]
        for l in I:
            if l not in i:
                noti.append(l)
        B=A[:,i];N=A[:,noti];Id=identity_matrix((A.ncols()-A.nrows()))
        bloque=copy(block_matrix([[-B^(-1)*N],[Id]]))
        print (bloque,'base=',i)

<p>Vamos a considerar las diferentes direcciones extremas obtenidas anteriormente, ordenadas adecuadamente y añadidas a una lista. Esto es, si tomamos las direcciones correspondientes a la "base =[0,2]", (es decir, $B=[\mathbf{a}_1, \mathbf{a}_3]$), la primera columna proporcina la dirección $(d_1,d_3,d_2,d_4)^t=(2,1,1,0)^t$ y la segunda columna $(d_1,d_3,d_2,d_4)^t=(1,1,0,1)^t$. Ordenarlas, supone escribirlas como: $(d_1,d_2,d_3,d_4)^t=(2,1,1,0)^t$ y $(d_1,d_2,d_3,d_4)^t=(1,0,1,1)^t$. Por tanto la lista de direcciones debidamente ordenadas es:</p>

In [ ]:
direcciones=[[2,1,1,0],[2,1,1,0],[1,0,1,1],[1,1/2,1/2,0],[1,0,1,1],[1,0,1,1]]

In [ ]:
matrizdirecciones=matrix(direcciones);rank(matrizdirecciones);show(matrizdirecciones)

<p>Aunque obtuvimos 6 expresiones sólo tenemos dos direcciones extremas, al estudiar el rango de dicha matriz. Observemos que dicha matriz ya está traspuesta (contiene los vectores dirección por filas). Las filas 1, 2 y 3 proporcionan la mismna dirección extrema: $(2,1,1,0)^t$, e igualmenta las filas 3, 5 y 6 la segunda dirección extrema: $(1, 0, 1, 1)^t$. Ambas ordenadas según la secuencia $(d_1,d_2,d_3,d_4)$. </p>
<h3>Conos y direcciones extremas</h3>
<p>Hemos obtenido el rango de la matriz formada por todas las direcciones, pero esto en ocasiones no nos proporciona las diferentes direcciones extremas. Para entenderlo necesitasmos al menos tres dimensiones, donde podemos visualizar un cono con cuatro rayos:</p>
<p>$$\{(2,1,0),(1,2,0),(2,1,1),(1,2,1)\}$$</p>
<p>Si consideramos estas cuatro direcciones y su subespacio vectorial, tendremos dimensión:</p>

In [ ]:
nuevamatrizdirecciones=matrix([[2,1,0],[1,2,0],[2,1,1],[1,2,1]])
nuevamatrizdirecciones.rank()

<p>Sin embargo generan un cono con cuatro direcciones extremas diferentes:</p>

In [ ]:
nuevocono=Cone([[2,1,0],[1,2,0],[2,1,1],[1,2,1]])

In [ ]:
nuevocono.rays()

<p>Repitiendo en el caso anterior, ahora con la estructura de cono, obtenemos:</p>

In [ ]:
cono=Cone(direcciones)

In [ ]:
cono.rays()

<p> </p>
<p>Si ahora estuviéramos interesados en tratar el problema:</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2)= x_1-x_2\\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>Estamos interesados en saber si tenemos o no solución a dicho problema. Hemos visto en teoría que si $c^t\vec{d}\geq 0$ para toda dirección extrema $\vec{d}$, el problema tiene solución y se alcanza en un punto extremo, así</p>

In [ ]:
c=vector([1,-1,0,0])

In [ ]:
matrizdirecciones*c

<p>Nuestro problema tiene solución y se alcanza en un punto extremo. Para calcular cuál, veamos cuál es el coste de los puntos extremos.</p>

In [ ]:
for i in powerset(I):
    if len(i)==2 and (((A[:,i]).det())!=0):
        print (A[:,i]\b, 'base=',i)

In [ ]:
extremos=[[2,4,0,0],[0,2,0,2],[0,0,2,6]]

In [ ]:
matrizextremos=matrix(extremos)

<p>Para obtener el valor de la función objetivo en cada uno de los puntos extremos, multiplicamos la matriz anterior por el vector de costes.</p>

In [ ]:
matrizextremos*c

<p>Así se alcanza el mínimo en $(2,4)$ y en $(0,2)$, por tanto tenemos infinitas soluciones (toda la arista que va entre dichos puntos extremos). Si hubiésemos consideramos la función de costes $Z(x_1,x_2)=x_1-3x_2$</p>

In [ ]:
c1=vector([1,-3,0,0])

In [ ]:
matrizdirecciones*c1

<p>Por lo tanto no tenemos solución acotada, podemos construir una sucesión de puntos factibles $x_n$ tales que</p>
<p>$$\lim_{n\to\infty}Z(x_n)=-\infty$$</p>
<h3>Programación Lineal Entera y Mixta</h3>
<p>SAGE tiene una clase especial <tt class="descname">MixedIntegerLinearProgram</tt> que nos permite definir un problema de Programación Lineal con variables que pueden ser enteras, continuas o que haya variables de ambos tipos (en ese caso se suele hablar de Programación Mixta). Si seguimos considerando nuestro problema del principio, con función objetivo $Z(x_1,x_2)=x_1-x_2$, es decir,</p>
<p>$$\left. \begin{array}{rl}\min & Z(x_1, x_2)= x_1-x_2\\ \mbox{s.t.} & - x_1 + x_2 \leq 2 \\&  - x_1 + 2 x_2 \leq 6 \\&  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>
<p>Lo que hacemos en primer lugar es asignar a <span style="font-family: courier new,courier;"><strong>p</strong></span> la clase <tt class="descname">MixedIntegerLinearProgram</tt>, por defecto se entiende que es un problema de maximización, si estamos en un caso como el nuestro de minimización, basta añadir la opción <span class="n">maximization</span><span class="o">=</span><span class="bp">False</span>:</p>

In [ ]:
p=MixedIntegerLinearProgram(maximization=False)

<p>Vamos a llamar a nuestras variables $x$ e $y$</p>

In [ ]:
x, y= p['x'], p['y']

<p>Para añadir restricciones, utilizamos el método <span class="n">add_constraint</span><span class="p">(</span>)</p>

In [ ]:
p.add_constraint(-x + y <= 2)
p.add_constraint(-x+2*y<=6)

<p>Para introducir la función de coste o función objetivo basta usar el método <span class="n">set_objective</span><span class="p">(</span>)</p>

In [ ]:
p.set_objective(x-y)

In [ ]:
p.show()

<p>Con la versión actual de SAGE las variables por defecto no son positivas, por lo que no estamos modelando el problema de inicio.</p>
<p>Resolvemos este problema de programación lineal con el método solve(), que nos proporcionará el valor objetivo mínimo (o máximo) y si queremos conocer el valor de las variables podemos usar get_values()</p>

In [ ]:
p.add_constraint(x>=0)
p.add_constraint(y>=0)
p.solve()

In [ ]:
p.get_values(x)

In [ ]:
p.get_values(y)

<p>Vamos ahora a considerar la misma región factible y cambiamos la función objetivo por $Z(x_1,x_2)=x_1-3x_2$, además introduciremos el método new_variable() que nos permite introducir un vector de variables, lo que puede ser muy útil a la hora de programar, y además nos permite añadir que dichas variables sean positivas, con la opción nonnegative=True. Aunque seguiremos la notación de clase SAGE permite el uso de cualquier nombre como variable. Por ejemplo podríamos poner "variablemia=p1.new_variable()"</p>

In [ ]:
p1=MixedIntegerLinearProgram(maximization=False)

In [ ]:
x=p1.new_variable(nonnegative=True)

In [ ]:
p1.add_constraint(-x[0] + x[1] <= 2)
p1.add_constraint(-x[0]+2*x[1]<=6)

In [ ]:
p1.set_objective(x[0]-3*x[1])

In [ ]:
p1.show()

In [ ]:
p1.solve()

<p>Habíamos visto anteriormente que este problema no estaba acotado, por eso hemos obtenido esta respuesta. En el caso que la región factible sea vacía la respuesta de SAGE será la misma.</p>
<h3>Modelos de programación matemática</h3>
<h4>Problema de la dieta</h4>
<p>El problema por excelencia de la programación lineal es el de optimización en la asignación de recursos. Un caso concreto es el conocido como el problema de la dieta. Consiste en determinar la composición de la dieta de mínimo coste que satisface la necesidades específicas de nutrientes.</p>
<p>Consideremos el siguiente ejemplo: Se ha determinado que las necesidades mínimas diarias en la alimentación de un ternero son de 700 gr de proteínas, 28 gr de Calcio y 150 mg de vitaminas. Los alimentos disponibles son pienso y forraje con un coste unitario de 0.30 y 0.35 €/kg respectivamente. La composición nutritiva por kg de alimento se muestra en la tabla siguiente:</p>
<table style="width: 244px; height: 61px; border-color: #000000; border-width: 1px; border-style: solid;" border="1" align="center">
<tbody>
<tr>
<td> </td>
<td>
<p>Proteinas(g)</p>
</td>
<td>Calcio (g)</td>
<td>Vitaminas (mg)</td>
</tr>
<tr>
<td>Pienso</td>
<td>30</td>
<td>2</td>
<td>10</td>
</tr>
<tr>
<td>Forraje </td>
<td>45</td>
<td>1</td>
<td>5</td>
</tr>
</tbody>
</table>
<p>Para modelar este problema consideraremos una variable $x_i$ para cada tipo de variable, que sea la cantidad diaria de kg de ese alimento que se considera. Tenemos un coste $c_i$ del Kg cada uno de estos alimentos. Nuestro problema tiene como función objetivo minimizar el coste de dicha alimentación es decir $\min\sum_{i=1}^2c_ix_i$. Además tenemos restricciones dadas por las necesidades de cada uno de los nutrientes considerados, si notamos $a_{ij}$ la cantidad del nutriente $j$ en el alimento $i$ y $b_j$ la cantidad mínima diaria del nutriente $j$ dichas restricciones son una para cada tipo de nutriente y son:</p>
<p>$$\sum_i a_{ij}x_i\geq b_j$$</p>
<p>Las necesidades de nutrientes no son siempre desigualdades ($\geq$) como la anterior, pueden ser igualdades o de tipo ($\leq$).</p>
<p>En nuestro ejemplo:</p>
<p>$$\left. \begin{array}{rl}\min &  0.30 x_1+0.35 x_2\\ \mbox{s.t.} & 30x_1 + 45 x_2 \geq 700 \\&  10 x_1 +  5 x_2 \geq 150 \\ &  2 x_1 +  x_2 \geq 28 \\ &  x_1 \geq 0, x_2 \geq 0 \end{array} \right\}$$</p>

In [ ]:
p2=MixedIntegerLinearProgram(maximization=False)

In [ ]:
x=p2.new_variable(nonnegative=True)

In [ ]:
p2.add_constraint(30*x[0] + 45*x[1] >= 700)
p2.add_constraint(10*x[0]+5*x[1]>=150)
p2.add_constraint(2*x[0]+x[1]>=28)

In [ ]:
p2.set_objective(0.30*x[0]+0.35*x[1])

In [ ]:
p2.solve()

In [ ]:
p2.get_values(x[0])

In [ ]:
p2.get_values(x[1])

También podemos pedir todas las soluciones:

In [ ]:
p2.get_values(x)

In [ ]:
NUMERO_ALUMNO=1

NOMBRE_FICHERO_EXAMEN = 'Ex_PL3_MASI_t.htl'
load('codigo_examinar_html.sage')

if NUMERO_ALUMNO > 0:
    lector_examenes(NOMBRE_FICHERO_EXAMEN,NUMERO_ALUMNO,False)